In [ ]:
# Kết nối Google Colab đến Google Drive của bạn:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# -> Example Input: https://www.youtube.com/watch?v=QmDDbqZ8VGc

# 3. Crawling and preprocessing text

3.1. Write a python program to crawl the first 200 comments of a video.

In [ ]:
videoURL = input("Vui lòng nhập địa chỉ URL của video muốn lấy comment: ")
print("Video định crawl là: " + str(videoURL))

Vui lòng nhập địa chỉ URL của video muốn lấy comment: https://www.youtube.com/watch?v=QmDDbqZ8VGc
Video định crawl là: https://www.youtube.com/watch?v=QmDDbqZ8VGc


In [ ]:
import requests

# Kiểm tra xem có video không
def try_site(url):
  try:
    request = requests.get(url, allow_redirects=False)
    return request.status_code == 200
  except:
    print("Đã xảy ra lỗi với URL")
    return False

try_site(str(videoURL))

True

In [ ]:
!pip install selenium # Dùng Selenium vì Youtube sử dụng JavaScript rendered, vốn là thứ nằm ngoài khả năng xử lý của BeautifulSoup
!apt-get update # để update ubuntu nhằm run apt install chính xác hơn
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin # Sao chép ChromeDriver sang /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') # Lưu ý là đang cấu hình cho ChromeDriver chạy được trên Google Colab; Nguồn tham khảo: https://colab.research.google.com/drive/1GFJKhpOju_WLAgiVPCzCGTBVGMkyAjtk
import time # Nhằm gọi sleep() để giúp chương trình đợi Youtube load các comment
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions # Cần những modules này là bởi comments trên Youtube được loaded động, chỉ hiện ra khi chúng ta lăn chuột xuống phần dưới của video.

     |████████████████████████████████| 968 kB 14.2 MB/s 
     |████████████████████████████████| 359 kB 67.5 MB/s 
     |████████████████████████████████| 138 kB 61.7 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 3.6 MB 57.9 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,831 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
H

In [ ]:
data = [] # Biến kết quả có dạng list, nó sẽ chứa các comments crawl về được dưới dạng các chuỗi String.

# Mở ChromeDriver lên và thực thi các lệnh sau:
with Chrome('chromedriver', options=chrome_options) as driver: 
  wait = WebDriverWait(driver, 15) # Chương trình sẽ chờ tối đa 15s để đợi element đạt được expected_conditions;
  # Khác với sleep(), Nếu chỉ với Ns (trong đó N < 15) mà element đã đạt expected_conditions 
  # thì chỉ cần sau Ns đó, thao tác tương ứng sẽ được thực hiện ngay
  # Lưu ý nếu quá 15s không thực thi xong thì sẽ bị lỗi
  driver.get(str(videoURL)) # Chương trình đi tới địa chỉ Url của video cần crawl
  
  # Ý tưởng chung là Dùng vòng lập để thực hiện: Lăn chuột -> đợi comment hiện lên và crawl nó

  # Dò hết comment có thể bắt gặp sau n lần lăn chuột 
  for item in range(11):
    wait.until(expected_conditions.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END) # Đợi đến khi thẻ body hiển thị thì thực hiện việc Lăn chuột xuống phía dưới video
    time.sleep(15) # Chờ cố định 15 milisecond; để tránh một số trường hợp chương trình chạy quá nhanh, chỉ wait không thì không hiệu quả nên cần thêm Sleep()
  
  try:
    # Lần lượt duyệt qua tất cả thẻ có id là content-text, đẩy comment có trong đó vào biến kết quả data
    for comment in wait.until(expected_conditions.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
      if(len(data)>200):
        break
      else:
        data.append(comment.text)
  except:
    print("Đã xảy ra lỗi")

In [ ]:
data[1:10]

['Hay xem lại cái này để nhớ lại những gì Bùi Tiến Dũng đã làm cho Việt Nam \nĐừng vì một lỗi sai mà quay lưng lại với thủ môn của chúng ta\nChán khi thấy mọi người gọi anh là thằng này đứa nọ',
 'Ông Cương vs ông Cường bình luận đến ném biên bên sân nhà  cũng hồi hộp =))',
 'xem đi xem lại vẫn thấy tự hào về đất nước xinh đẹp mang tên Việt Nam. I want to say, I love Việt Nam',
 'Dù đội tuyển hiện giờ có như nào đi nữa.. nhưng ai cũng nhớ vào những ngày tháng năm đó.. có 1 u23 Việt Nam.....,,!!!',
 'Xem lại biết bao lần mà cảm xúc vẫn dâng trào như ngày nào luôn, cảm ơn ad đã chia sẻ',
 '04/12/2019 \nKhi xem lại thì vẫn không thể giấu đi được cảm xúc, chính nhờ lứa U23 này đã khiến tôi có niềm tin hơn vào nền bóng đá VN. Và bây giờ đang là Sea Games 30 tôi mong U22 VN cũng sẽ làm rạng danh đất nước. Cảm ơn các bạn thật nhiều ',
 'Ai vừa xem lại khi Việt Nam vào chung kết AFF không, cảm xúc vẫn ùa về như vừa xem trực tiếp xong.',
 '10/12/2019 tấm HCV Seagame 30 với thành tích bất bại sa

In [ ]:
# Xem thử 5 hàng đầu của dữ liệu crawl về
for i in range(5):
  print("Dòng " + str(i+1) + ": ", data[i])

Dòng 1:  Thấy nhiều bạn hỏi bài nhạc mình cho các bạn tên bài luôn nha:
1.  Moonlight sonata-beethoven
2. Epic legend
3. Breath and life
Dòng 2:  Hay xem lại cái này để nhớ lại những gì Bùi Tiến Dũng đã làm cho Việt Nam 
Đừng vì một lỗi sai mà quay lưng lại với thủ môn của chúng ta
Chán khi thấy mọi người gọi anh là thằng này đứa nọ
Dòng 3:  Ông Cương vs ông Cường bình luận đến ném biên bên sân nhà  cũng hồi hộp =))
Dòng 4:  xem đi xem lại vẫn thấy tự hào về đất nước xinh đẹp mang tên Việt Nam. I want to say, I love Việt Nam
Dòng 5:  Dù đội tuyển hiện giờ có như nào đi nữa.. nhưng ai cũng nhớ vào những ngày tháng năm đó.. có 1 u23 Việt Nam.....,,!!!


In [ ]:
# Ghi dữ liệu crawl về được ra file csv:
import pandas as pd 
dataset_path = "/content/gdrive/MyDrive/libPy/youtube_comment_dataset.csv" # Đường dẫn đến file dữ liệu dataset trên Google Drive

df = pd.DataFrame(data, columns=['Comment'])
df.to_csv(dataset_path)
print(df.head())

                                             Comment
0  Thấy nhiều bạn hỏi bài nhạc mình cho các bạn t...
1  Hay xem lại cái này để nhớ lại những gì Bùi Ti...
2  Ông Cương vs ông Cường bình luận đến ném biên ...
3  xem đi xem lại vẫn thấy tự hào về đất nước xin...
4  Dù đội tuyển hiện giờ có như nào đi nữa.. nhưn...


3.2. Apply text preprocessing techniques and TF-IDF for feature extraction of the first 200 comments.

In [ ]:
# Kiểm tra đơn giản thử xem đã có file dữ liệu hay chưa:
import os

import pandas as pd
dataset_path = "/content/gdrive/MyDrive/libPy/youtube_comment_dataset.csv" # Đường dẫn đến file dữ liệu dataset trên Google Drive

if(os.path.exists(dataset_path) and os.path.isfile(dataset_path)):
  file_check = pd.read_csv(dataset_path)
  print(file_check)
else:
  print("Không tìm thấy Dataset!")

     Unnamed: 0                                            Comment
0             0  Thấy nhiều bạn hỏi bài nhạc mình cho các bạn t...
1             1  Hay xem lại cái này để nhớ lại những gì Bùi Ti...
2             2  Ông Cương vs ông Cường bình luận đến ném biên ...
3             3  xem đi xem lại vẫn thấy tự hào về đất nước xin...
4             4  Dù đội tuyển hiện giờ có như nào đi nữa.. nhưn...
..          ...                                                ...
195         195                                       Đầy cảm xúc 
196         196                            Mạnh mẽ lên các anh ️️️
197         197  Bị loại ngay vòng ngoài của SeaGame 29 mà lại ...
198         198  Đoạn Công Phượng trả lời phỏng vấn mà thương ghê.
199         199                           Xem lại vẫn xúc động vl 

[200 rows x 2 columns]


In [ ]:
# Đọc dữ liệu từ file dataset
import csv
comments_list = [] # Một biến list chứa các comment dưới dạng string

with open(dataset_path, newline='') as inputfile:
    for row in csv.reader(inputfile):
      if(len(comments_list)<201):
        comments_list.append(row[1])
      else:
        break

comments_list.pop(0) # Xóa nhãn label
print("Số lượng comment: ", len(comments_list))
print("comments = ", comments_list[0:10])

Số lượng comment:  200
comments =  ['Thấy nhiều bạn hỏi bài nhạc mình cho các bạn tên bài luôn nha:\n1.  Moonlight sonata-beethoven\n2. Epic legend\n3. Breath and life', 'Hay xem lại cái này để nhớ lại những gì Bùi Tiến Dũng đã làm cho Việt Nam \nĐừng vì một lỗi sai mà quay lưng lại với thủ môn của chúng ta\nChán khi thấy mọi người gọi anh là thằng này đứa nọ', 'Ông Cương vs ông Cường bình luận đến ném biên bên sân nhà  cũng hồi hộp =))', 'xem đi xem lại vẫn thấy tự hào về đất nước xinh đẹp mang tên Việt Nam. I want to say, I love Việt Nam', 'Dù đội tuyển hiện giờ có như nào đi nữa.. nhưng ai cũng nhớ vào những ngày tháng năm đó.. có 1 u23 Việt Nam.....,,!!!', 'Xem lại biết bao lần mà cảm xúc vẫn dâng trào như ngày nào luôn, cảm ơn ad đã chia sẻ', '04/12/2019 \nKhi xem lại thì vẫn không thể giấu đi được cảm xúc, chính nhờ lứa U23 này đã khiến tôi có niềm tin hơn vào nền bóng đá VN. Và bây giờ đang là Sea Games 30 tôi mong U22 VN cũng sẽ làm rạng danh đất nước. Cảm ơn các bạn thật nhiều

In [ ]:
import regex as re

# Xóa code HTML có trong dữ liệu
def remove_html(text):
  return re.sub(r'<[^>]*>', '', text)

def remove_html_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(remove_html(C_list[i]))
  return result

# Ví dụ
html_example = ['<p class=\"par\">Ví dụ 1</p>', '<h>Ví dụ 2<h><br>']
print(remove_html_main(html_example))

['Ví dụ 1', 'Ví dụ 2']


In [ ]:
import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
  dic = {}
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
  charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
  for i in range(len(char1252)):
    dic[char1252[i]] = charutf8[i]
  return dic
  
dicchar = loaddicchar()
# Chuẩn hóa bảng mã tiếng Việt
def covert_unicode(text):
  return re.sub(r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ', 
                lambda x: dicchar[x.group()], text)

def covert_unicode_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(covert_unicode(C_list[i]))
  return result

# Ví dụ:
unicode_example = ['Thái', 'Thái']
print(unicode_example[0] == unicode_example[1])
print(unicode_example[0] == covert_unicode(unicode_example[1]))
print(len(unicode_example[0]), ":", len(unicode_example[1]))

False
True
4 : 5


In [ ]:
# Word Tokenize
def word_tokenize_list(C_list):
  result = []
  for i in range(len(C_list)):
    x = word_tokenize(C_list[i])
    for j in range(len(x)):
      result.append(x[j])
  return result

In [ ]:
# Đổi tất cả từ về dạng chữ thường
def all_word_lowercase_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(C_list[i].lower())
  return result

In [ ]:
import regex as re

# Xóa các biểu tượng đặc biệt
def strip_emoji(text):
  EMOJI_PATTERN = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
  )
  text = re.sub(EMOJI_PATTERN, r'', text)
  return text

def strip_emoji_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(strip_emoji(C_list[i]))
  return result

emoji_example = ["😍Hello😍", "Pros🙏trate"]
print(strip_emoji_main(emoji_example))

['Hello', 'Prostrate']


In [ ]:
# Stopword
stopword_path = "/content/gdrive/MyDrive/libPy/vietnamese-stopwords.txt" # Đường dẫn đến file dữ liệu stopword trên Google Drive
with open(stopword_path, 'r') as file:
  raw = file.read().replace('\n', ';')
  STOPWORDS = raw.split(";")

def remove_stopword_main(C_list):
  result = []
  for i in range(len(C_list)):
    if C_list[i] not in STOPWORDS:
      result.append(C_list[i])
  return result

print(STOPWORDS[0:5])

['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy']


In [ ]:
# Xóa dấu câu
daucau = '''!()-[]{};:'"\,<>./?@#$%^&*_~...='''

def remove_punctutation_list(C_list):
  result = []
  for i in range(len(C_list)):
    for j in C_list[i]:
      if j in daucau:
        C_list[i] = C_list[i].replace(j, "")
    
    result.append(C_list[i])
  return result

# Ví dụ
punctutation_example = ["ví@# !$%dụ =1=", "ví dụ &2"]
print(remove_punctutation_list(punctutation_example))

['ví dụ 1', 'ví dụ 2']


In [ ]:
# Xóa các số có trong list
def number_remove_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(re.sub(r'\d+', '', C_list[i]))
  return result

# Ví dụ
num_example = ["123456", "0U0", "Hel7lo"]
print(number_remove_main(num_example))

['', 'U', 'Hello']


In [ ]:
# Xóa khoảng trắng thừa
def extra_whitespace_remove_main(C_list):
  result = []
  for i in range(len(C_list)):
    result.append(re.sub(r'\s+', ' ', C_list[i].strip()))
  return result

# Ví dụ
whitespace_example = [" Ví dụ 1 ", "Ví   dụ   2", "   Ví dụ   3"]
print(extra_whitespace_remove_main(whitespace_example))

['Ví dụ 1', 'Ví dụ 2', 'Ví dụ 3']


In [ ]:
# Xóa các chuỗi rỗng trong list dữ liệu dataset
def remove_emptyStr(C_list):
  result = []
  for i in range(len(C_list)):
    if (C_list[i]!='' and C_list[i]!=' ' and C_list[i]!='️' and C_list[i]!='️ ️' and C_list[i]!='️ ️ ️' and C_list[i]!='️ ️ ️ ️ ️'):
      result.append(C_list[i])
  return result

# Ví dụ
EmpStr_example = ['', 'Ví dụ 2', '️']
print(EmpStr_example[0] == EmpStr_example[1])
print(len(EmpStr_example[0]), ":", len(EmpStr_example[1]))
print(remove_emptyStr(EmpStr_example))

False
0 : 7
['Ví dụ 2']


In [ ]:
!pip install underthesea # cài đặt Thư viện NLP dành cho tiếng Việt

     |████████████████████████████████| 7.6 MB 11.6 MB/s 
     |████████████████████████████████| 581 kB 79.0 MB/s 
     |████████████████████████████████| 235 kB 80.3 MB/s 
     |████████████████████████████████| 965 kB 82.9 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from underthesea import word_tokenize

def text_preprocess(document):
  com_list1 = remove_html_main(document) # Xóa các phần tử HTML nếu có
  com_list2 = covert_unicode_main(com_list1) # Chuẩn hóa các từ để máy không bị nhầm lẫn
  com_list3 = word_tokenize_list(com_list2) # Tách từ trong câu
  com_list4 = all_word_lowercase_main(com_list3) # Chuyển tất cả về chữ thường
  com_list5 = strip_emoji_main(com_list4) # Xóa các ký tự đặc biệt nếu có
  com_list6 = remove_stopword_main(com_list5) # Xóa các từ không quan trọng trong tiếng Việt
  com_list7 = remove_punctutation_list(com_list6) # Xóa các dấu câu
  com_list8 = number_remove_main(com_list7) # Xóa số khỏi chuỗi
  com_list9 = extra_whitespace_remove_main(com_list8) # Xóa các khoảng trắng liên tục trong câu nếu có
  com_list10 = remove_emptyStr(com_list9) # Xóa các chuỗi rỗng
  return com_list10

new_comments_list = text_preprocess(comments_list)
print(new_comments_list)
print("Length: ", len(new_comments_list))

['nhạc', 'nha', 'moonlight', 'sonatabeethoven', 'epic', 'legend', 'breath', 'and', 'life', 'bùi tiến dũng', 'việt nam', 'đừng', 'lỗi', 'sai', 'lưng', 'thủ môn', 'chán', 'gọi', 'thằng', 'đứa', 'cương', 'vs', 'cường', 'bình luận', 'ném biên', 'sân nhà', 'hồi hộp', 'đi', 'tự hào', 'đất nước', 'xinh đẹp', 'việt nam', 'i', 'want', 'to', 'say', 'i', 'love', 'việt nam', 'đội tuyển', 'hiện giờ', 'đi', 'u', 'việt nam', 'cảm xúc', 'dâng', 'trào', 'ad', 'giấu', 'đi', 'cảm xúc', 'lứa', 'u', 'niềm', 'bóng đá', 'vn', 'sea games', 'mong', 'u', 'vn', 'rạng danh', 'đất nước', 'cảm ơn', 'việt nam', 'chung kết', 'aff', 'cảm xúc', 'ùa', 'xong', 'hcv', 'seagame', 'thành tích', 'bất', 'bại', 'chờ đợi', 'hành trình', 'nước mắt', 'rơi', 'ngàn', 'vạn', 'chân thành', 'gởi', 'hlv', 'park', 'hang', 'seo', 'thầy', 'park', 'bóng đá', 'vn', 'nổi', 'da gà', 'rơi', 'nước mắt', 'nổi', 'da gà', 'tự hào', 'việt nam', 'cám ơn', 'park hangseo', 'việt nam', 'cám ơn', 'huấn luyện', 'con người', 'tài năng', 'kia', 'bộc lộ', '

In [ ]:
# Hàm để Convert "list" thành "list of lists"
def List2LOLs(lst):
    return [[element] for element in lst]

new_comments_list2 = List2LOLs(new_comments_list)
print(new_comments_list2[0:10])
print("Length: ", len(new_comments_list2))

[['nhạc'], ['nha'], ['moonlight'], ['sonatabeethoven'], ['epic'], ['legend'], ['breath'], ['and'], ['life'], ['bùi tiến dũng']]
Length:  2121


In [ ]:
# TF-IDF: 
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy(doc):
  return doc

vectorizer2 = TfidfVectorizer(analyzer='word', tokenizer=dummy, preprocessor=dummy, token_pattern=None) # Khai báo
vectorizer2.fit(new_comments_list2) # Build từ điển
print("Length: ", len(vectorizer2.vocabulary_))
#print(vectorizer2.idf_)

Length:  776


In [ ]:
count = 0
vocal = vectorizer2.vocabulary_
for value in sorted(vocal):
  if(count > 30):
    break
  else:
    key = vocal[value]
    count += 1
    print(key, ": ", value)

0 :  a
1 :  ad
2 :  aff
3 :  aff cup
4 :  ai
5 :  ak
6 :  and
7 :  anh hùng
8 :  asiad
9 :  asiad games
10 :  asian cup
11 :  ban đầu
12 :  banh
13 :  bao
14 :  bdvn
15 :  bh
16 :  bi tráng
17 :  bik
18 :  biên tập
19 :  bn
20 :  breath
21 :  btd
22 :  bus
23 :  buồn
24 :  bàn
25 :  bàn tay
26 :  bán kết
27 :  bây h
28 :  bãn
29 :  bình luận
30 :  bóng
